# Experiments with PreXIA on GLUE

## Installation

This is for the legacy code used by original prexia

In [ ]:
#@title 1. Keep this tab alive to prevent Colab from disconnecting you { display-mode: "form" }

#@markdown Press play on the music player that will appear below:
%%html
<audio src="https://oobabooga.github.io/silence.m4a" controls>

Next, we install the required packages.

In [ ]:
#!pip install cloud-tpu-client==0.10 torch==2.0.0 torchvision==0.15.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl
#!pip install adapter-transformers
#!pip install torch==2.0


!pip uninstall -y transformers tokenizers huggingface_hub

!pip install -U adapter-transformers
!pip install huggingface_hub
!pip install datasets
!pip install evaluate

Found existing installation: transformers 4.40.2
Uninstalling transformers-4.40.2:
  Successfully uninstalled transformers-4.40.2
Found existing installation: tokenizers 0.19.1
Uninstalling tokenizers-0.19.1:
  Successfully uninstalled tokenizers-0.19.1
Found existing installation: huggingface-hub 0.20.3
Uninstalling huggingface-hub-0.20.3:
  Successfully uninstalled huggingface-hub-0.20.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Transformers version check

1.   List item
2.   List item



Should be 4.26.1 for PreXIA

In [ ]:
import transformers, torch
print(transformers.__version__,torch.__version__)

4.26.1 2.2.1+cu121


## (Optionally) Login to Huggingface

If you dont have a token saved, uncomment the following two lines and run them

In [ ]:
#from huggingface_hub import notebook_login

#notebook_login()

## PreXIA

Thefollowing block contains the code used to create PreXIA in COLING paper.

```python
from transformers.adapters import ConfigUnion, ParallelConfig, IA3Config, PrefixTuningConfig, CompacterPlusPlusConfig

config = ConfigUnion(
    IA3Config(use_gating=True, alpha=4.0),
    PrefixTuningConfig(prefix_length=30, bottleneck_size=768, use_gating=True),
    ParallelConfig(use_gating=True)
)

model.add_adapter("prexia", config=config)

model.add_classification_head(
    f"prexia",
    num_labels=2,
  )

model.set_active_adapters( "prexia")

model.train_adapter('prexia')j
```

# The follwing code is used to evaluate PreXIA on GLUE

In [ ]:
TASK_NAME = "qnli"


GLUE_METRICS = {
    'cola': 'matthews_correlation',
    'sst2': 'accuracy',
    'mrpc': 'f1',
    'qqp': 'accuracy',
    'stsb': 'pearsonr',
    'mnli': 'accuracy',
    'qnli': 'accuracy',
    'rte': 'accuracy',
    'wnli': 'accuracy',
}
# base model name. Some use bass some large
MODEL_NAME = "bert-base-cased"

# number of samples to use from the  set
K = 10000


import dataclasses
import os
import sys
from dataclasses import dataclass, field
from typing import Dict, Optional

import numpy as np


from transformers import AdapterTrainer
from transformers import BertConfig, BertTokenizer,AutoConfig, AutoModelForSequenceClassification, AutoTokenizer, EvalPrediction, GlueDataset
from transformers import GlueDataTrainingArguments as DataTrainingArguments
from transformers import (
    HfArgumentParser,
    TrainingArguments,
    glue_output_modes,
    glue_compute_metrics,
    glue_tasks_num_labels,
    set_seed,
)
from transformers.adapters import BertModelWithHeads, ConfigUnion, IA3Config, ParallelConfig, PrefixTuningConfig
import evaluate
from evaluate import load
from datasets import load_dataset



def get_sentence_keys():
  task_to_keys = {
    "cola": ("sentence", None),
    "mnli": ("premise", "hypothesis"),
    "mnli-mm": ("premise", "hypothesis"),
    "mrpc": ("sentence1", "sentence2"),
    "qnli": ("question", "sentence"),
    "qqp": ("question1", "question2"),
    "rte": ("sentence1", "sentence2"),
    "sst2": ("sentence", None),
    "stsb": ("sentence1", "sentence2"),
    "wnli": ("sentence1", "sentence2"),
  }

  s1, s2 = task_to_keys[datasets_task_name()]
  return s1, s2

def datasets_task_name():
  return TASK_NAME.replace('-','')

def get_metric_name():
  return GLUE_METRICS[datasets_task_name()]


def get_metric_for_task():
  return load('glue', get_metric_name())

def get_output_dir():
  return os.path.join(
        "/content/output/",
        f"prexia-{TASK_NAME}")

def get_output_mode():
  # Is this task classification or regression
  output_mode = glue_output_modes[TASK_NAME]
  return output_mode

def get_num_labels():
  return glue_tasks_num_labels[TASK_NAME]

OUTPUT_DIR = get_output_dir()
""" Finetuning the library models for sequence classification on
GLUE (Bert, XLM, XLNet, RoBERTa, Albert, XLM-RoBERTa)."""



!mkdir /content/output in



print("GLUE Task:",TASK_NAME,
      get_output_mode(),
      get_num_labels())


def get_metric_name():
    # Dynamic retrieval of metric name based on the dataset task name
    return GLUE_METRICS[datasets_task_name()]

def get_metric_for_task():
    # Load the appropriate metric from the evaluate library
    metric_name = get_metric_name()
    return load(metric_name)

mkdir: cannot create directory ‘/content/output’: File exists
mkdir: cannot create directory ‘in’: File exists
GLUE Task: qnli classification 2


In [ ]:
encoded_dataset["validation_matched"]

Dataset({
    features: ['labels', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 9815
})

In [ ]:

def configure_model():
  # Find out home many labels we must use for this task
  num_labels = get_num_labels()

  bert_config = BertConfig.from_pretrained(
    MODEL_NAME,
    finetuning_task=TASK_NAME,
    num_labels=num_labels,
    hidden_dropout_prob=0.05,  # Doropout probability for the hidden layers
    attention_probs_dropout_prob=0.05  # Dropout probability for the attention probabilities
  )
  model = BertModelWithHeads.from_pretrained(
    MODEL_NAME,
    config=bert_config,
  )
  tokenizer = BertTokenizer.from_pretrained(
    MODEL_NAME,
    truncation=True,
    padding='max_length',
    max_length=256)

  config = ConfigUnion(
      IA3Config(use_gating=True),
      PrefixTuningConfig(prefix_length=30, bottleneck_size=800, use_gating=True),
      ParallelConfig(use_gating=True)
  )

  model.add_adapter("prexia", config=config)

  model.add_classification_head(
      f"prexia",
      num_labels=num_labels,

  )

  model.set_active_adapters( "prexia")

  model.train_adapter('prexia')
  return model, tokenizer




def get_dataset():
  ds = load_dataset("glue", datasets_task_name())

  if K == "all":
    k = len(ds["train"])
  else:
    k = K

    dev_k = ds["train"].train_test_split(test_size=k, seed=42)
    ds["train"] = dev_k['test']
    assert k == len(ds["train"]), "Training set must be of size K!"
  return ds

def encode_dataset(dataset, tokenizer):
  sentence1_key, sentence2_key = get_sentence_keys()

  def preprocess_function(examples):
    if sentence2_key is None:
      return tokenizer(examples[sentence1_key], truncation=True, padding='max_length', max_length=256)
    return tokenizer(examples[sentence1_key], examples[sentence2_key], truncation=True, padding='max_length', max_length=256)

  encoded = dataset.map(preprocess_function, batched=True)

  if sentence2_key is not None:
    encoded = encoded.remove_columns([sentence1_key, sentence2_key])
  else:
    encoded = encoded.remove_columns([sentence1_key])
  encoded = encoded.rename_column("label", "labels")
  encoded.set_format("torch")
  return encoded

dataset = get_dataset()

model, tokenizer = configure_model()

encoded_dataset = encode_dataset(dataset, tokenizer)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--bert-base-cased/snapshots/cd5ef92a9fb2f889e972770a36d4ed042daf221e/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.05,
  "classifier_dropout": null,
  "finetuning_task": "qnli",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.05,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
 

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.
Be aware, overflowing tokens are not returned for the setting you have chosen, i.e. sequence pairs with the 'longest_first' truncation strategy. So the returned list will always be empty even if some tokens have been removed.


In [ ]:


def configure_trainer(model, tokenizer, encoded_dataset):
    output_mode = get_output_mode()
    num_labels = get_num_labels()
    metric = get_metric_for_task()
    metric_name = get_metric_name()

    def compute_metrics(p: EvalPrediction) -> Dict:
        preds = np.argmax(p.predictions, axis=1) if output_mode == "classification" else np.squeeze(p.predictions)
        return metric.compute(predictions=preds, references=p.label_ids)




    training_args = TrainingArguments(
        learning_rate=8e-5,
        warmup_ratio=0.1,
        weight_decay=0.001,
        adam_epsilon=1e-6,
        max_grad_norm=1.0,
        num_train_epochs=25,
        per_device_train_batch_size=128,
        per_device_eval_batch_size=128,
        logging_strategy="epoch",
        output_dir=OUTPUT_DIR,
        logging_dir=os.path.join(OUTPUT_DIR, "logs"),
        overwrite_output_dir=True,
        fp16=True,
        remove_unused_columns=False,
        evaluation_strategy="epoch",
        save_strategy="epoch",
        save_total_limit=5,
        #lr_scheduler="linear",
        dataloader_drop_last=True,
        gradient_accumulation_steps=2,
        label_smoothing_factor=0.1 if output_mode == "classification" else 0,
        load_best_model_at_end=True,
        metric_for_best_model=metric_name,
        greater_is_better=True if metric_name != "matthews_correlation" else False,
    )

    trainer = AdapterTrainer(
        model=model,
        args=training_args,
        train_dataset=encoded_dataset["train"],
        eval_dataset=encoded_dataset["validation"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics
    )

    return trainer

In [ ]:
trainer = configure_trainer(model, tokenizer, encoded_dataset)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Using cuda_amp half precision backend
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10000
  Num Epochs = 25
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 2
  Total optimization steps = 975
  Number of trainable parameters = 23187550


Epoch,Training Loss,Validation Loss,Accuracy
1,0.697000,0.689900,0.496652
2,0.681000,0.641793,0.661272
3,0.594700,0.489710,0.811198
4,0.505700,0.463092,0.824219
5,0.433200,0.436034,0.853609
6,0.380900,0.446521,0.838914
7,0.337400,0.459603,0.850074
8,0.292300,0.465337,0.853795
9,0.266100,0.498863,0.848958
10,0.250000,0.510731,0.848400


***** Running Evaluation *****
  Num examples = 5463
  Batch size = 128
Saving model checkpoint to /content/output/prexia-qnli/checkpoint-39
Configuration saved in /content/output/prexia-qnli/checkpoint-39/prexia/adapter_config.json
Module weights saved in /content/output/prexia-qnli/checkpoint-39/prexia/pytorch_adapter.bin
Configuration saved in /content/output/prexia-qnli/checkpoint-39/prexia/head_config.json
Module weights saved in /content/output/prexia-qnli/checkpoint-39/prexia/pytorch_model_head.bin
Configuration saved in /content/output/prexia-qnli/checkpoint-39/prexia/head_config.json
Module weights saved in /content/output/prexia-qnli/checkpoint-39/prexia/pytorch_model_head.bin
tokenizer config file saved in /content/output/prexia-qnli/checkpoint-39/tokenizer_config.json
Special tokens file saved in /content/output/prexia-qnli/checkpoint-39/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 5463
  Batch size = 128
Saving model checkpoint to /content/output

KeyboardInterrupt: 

In [ ]:
trainer.save_model()

Saving model checkpoint to /content/output/prexia-sst-2
Configuration saved in /content/output/prexia-sst-2/prexia/adapter_config.json
Module weights saved in /content/output/prexia-sst-2/prexia/pytorch_adapter.bin
Configuration saved in /content/output/prexia-sst-2/prexia/head_config.json
Module weights saved in /content/output/prexia-sst-2/prexia/pytorch_model_head.bin
Configuration saved in /content/output/prexia-sst-2/prexia/head_config.json
Module weights saved in /content/output/prexia-sst-2/prexia/pytorch_model_head.bin
tokenizer config file saved in /content/output/prexia-sst-2/tokenizer_config.json
Special tokens file saved in /content/output/prexia-sst-2/special_tokens_map.json


In [ ]:

tokenizer.save_pretrained(OUTPUT_DIR)

tokenizer config file saved in /content/output/prexia-sst-2/tokenizer_config.json
Special tokens file saved in /content/output/prexia-sst-2/special_tokens_map.json


('/content/output/prexia-sst-2/tokenizer_config.json',
 '/content/output/prexia-sst-2/special_tokens_map.json',
 '/content/output/prexia-sst-2/vocab.txt',
 '/content/output/prexia-sst-2/added_tokens.json')

In [ ]:


output_mode = get_output_mode()
p = trainer.predict(test_dataset=encoded_dataset["test"])
print(p)

***** Running Prediction *****
  Num examples = 1821
  Batch size = 32


PredictionOutput(predictions=array([[ 1.576 , -1.7295],
       [ 1.689 , -1.728 ],
       [-1.425 ,  1.448 ],
       ...,
       [ 0.2524, -0.3608],
       [ 1.026 , -1.336 ],
       [-1.221 ,  1.241 ]], dtype=float16), label_ids=array([-1, -1, -1, ..., -1, -1, -1]), metrics={'test_loss': 1.5171172618865967, 'test_accuracy': 0.0, 'test_runtime': 3.2263, 'test_samples_per_second': 564.431, 'test_steps_per_second': 17.668})


In [ ]:

results = {}

if output_mode == "classification":
  predictions = np.argmax(p.predictions, axis=1)
else:
  predictions = np.squeeze(p.predictions)
output_test_file = os.path.join(
      OUTPUT_DIR, f"test_results_{TASK_NAME}.txt"
  )

with open(output_test_file, "w") as writer:
  writer.write("index\tprediction\n")

  for index, item in enumerate(predictions):
    if output_mode == "regression":
      writer.write("%d\t%3.3f\n" % (index, item))
    else:
      writer.write("%d\t%s\n" % (index, item))

    # Convert the predictions array into a list of key-value pairs
    results.update([(index, item)])

In [ ]:
from sklearn. metrics import f1_score

preds = [results[i] for i in range(len(results.values()))]
gt = dataset["test"]["label"][:len(preds)]
f1_score(gt, preds)


0.8565273588970271

{'f1': 0.8565273588970271}


In [ ]:
To further enhance the performance of UniPELT, several additional adapter designs could be integrated. These designs offer unique mechanisms for parameter-efficient fine-tuning and could be beneficial when combined within the UniPELT framework.

### 1. **AdapterFusion**
AdapterFusion combines multiple task-specific adapters to create a unified adapter that can generalize better across tasks. By learning to fuse the outputs of different adapters, AdapterFusion can enhance model performance in multi-task scenarios.

**Example Configuration:**
```python
from adapters import AdapterFusionConfig, AutoAdapterModel

model = AutoAdapterModel.from_pretrained("roberta-base")

# Configure AdapterFusion
fusion_config = AdapterFusionConfig(adapter_names=["adapter1", "adapter2"])

# Add AdapterFusion to the model
model.add_adapter("adapter_fusion", config=fusion_config)

# Activate and train the adapter
model.train_adapter("adapter_fusion")
```

### 2. **Prompt Tuning**
Prompt Tuning involves training a small set of continuous task-specific prompts to guide the model's outputs. It can be particularly effective for few-shot learning scenarios.

**Example Configuration:**
```python
from adapters import PromptTuningConfig, AutoAdapterModel

model = AutoAdapterModel.from_pretrained("roberta-base")

# Configure Prompt Tuning
prompt_config = PromptTuningConfig(prompt_length=10)

# Add Prompt Tuning to the model
model.add_adapter("prompt_tuning", config=prompt_config)

# Activate and train the adapter
model.train_adapter("prompt_tuning")
```

### 3. **(IA)^3 (Infused Adapter by Inhibiting and Amplifying Inner Activations)**
(IA)^3 modifies the internal activations of the model layers by selectively inhibiting or amplifying certain activations, allowing for fine-grained control over the model's behavior.

**Example Configuration:**
```python
from adapters import IA3Config, AutoAdapterModel

model = AutoAdapterModel.from_pretrained("roberta-base")

# Configure (IA)^3
ia3_config = IA3Config()

# Add (IA)^3 to the model
model.add_adapter("ia3_adapter", config=ia3_config)

# Activate and train the adapter
model.train_adapter("ia3_adapter")
```

### 4. **Compacter**
Compacter uses low-rank hypercomplex layers to reduce the number of parameters while maintaining performance. It is effective in reducing the computational footprint of the model.

**Example Configuration:**
```python
from adapters import CompacterConfig, AutoAdapterModel

model = AutoAdapterModel.from_pretrained("roberta-base")

# Configure Compacter
compacter_config = CompacterConfig()

# Add Compacter to the model
model.add_adapter("compacter_adapter", config=compacter_config)

# Activate and train the adapter
model.train_adapter("compacter_adapter")
```

### 5. **Prefix Tuning with Dynamic Adjustment**
This method involves dynamically adjusting the prefix lengths during training based on task requirements, offering flexibility and improved performance.

**Example Configuration:**
```python
from adapters import PrefixTuningConfig, AutoAdapterModel

model = AutoAdapterModel.from_pretrained("roberta-base")

# Configure Prefix Tuning with dynamic adjustment
prefix_tuning_config = PrefixTuningConfig(prefix_length=20, dynamic=True)

# Add Prefix Tuning to the model
model.add_adapter("prefix_tuning_dynamic", config=prefix_tuning_config)

# Activate and train the adapter
model.train_adapter("prefix_tuning_dynamic")
```

### Integration into UniPELT:
By integrating these diverse adapter designs, UniPELT can benefit from the strengths of each method, resulting in a more robust and versatile fine-tuning framework.

**Example Integration:**
```python
from adapters import (
    AdaptivePrefixConfig,
    HierBnConfig,
    QLoRAConfig,
    AdapterFusionConfig,
    PromptTuningConfig,
    IA3Config,
    CompacterConfig,
    AutoAdapterModel
)

model = AutoAdapterModel.from_pretrained("roberta-base")

# Initialize and configure adapters
prefix_config = AdaptivePrefixConfig(prefix_length=30, dynamic=True)
hier_bn_config = HierBnConfig(levels=3, reduction_factor=16)
qlora_config = QLoRAConfig(r=8, alpha=16, quantization_bits=4)
fusion_config = AdapterFusionConfig(adapter_names=["adaptive_prefix", "hierarchical_bottleneck", "qlora_adapter"])
prompt_config = PromptTuningConfig(prompt_length=10)
ia3_config = IA3Config()
compacter_config = CompacterConfig()

# Add adapters to the model
model.add_adapter("adaptive_prefix", config=prefix_config)
model.add_adapter("hierarchical_bottleneck", config=hier_bn_config)
model.add_adapter("qlora_adapter", config=qlora_config)
model.add_adapter("adapter_fusion", config=fusion_config)
model.add_adapter("prompt_tuning", config=prompt_config)
model.add_adapter("ia3_adapter", config=ia3_config)
model.add_adapter("compacter_adapter", config=compacter_config)

# Activate and train adapters
model.train_adapter(["adaptive_prefix", "hierarchical_bottleneck", "qlora_adapter", "adapter_fusion", "prompt_tuning", "ia3_adapter", "compacter_adapter"])
```

### Benefits:
1. **Versatility:** Incorporates multiple fine-tuning strategies to handle various tasks efficiently.
2. **Performance:** Each adapter method contributes to overall model performance, improving generalization and task-specific accuracy.
3. **Efficiency:** Methods like QLoRA and Compacter ensure parameter efficiency and reduced computational overhead.

By leveraging these additional adapter designs, UniPELT can be further enhanced to deliver superior performance across a wide range of NLP tasks.